<a href="https://colab.research.google.com/github/vyphamhung10/khoa_luan/blob/master/price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""working.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1llsccnklCzs7EZSELP6MVysncixaZnQR

## Notebook settings
"""
# region Import
# Data download
# Import basic
import csv
import math
import os
import warnings
# Init google drive
# from google.colab import drive
from datetime import datetime
from timeit import default_timer as timer

import numpy as np
import pandas as pd
# Plottool
import plotly.graph_objs as go
# IPython
from IPython.display import display
# Hyperopt bayesian optimization
from hyperopt import hp, Trials, tpe, fmin, STATUS_OK, partial
# Keras
from keras import Sequential
from keras import optimizers
from keras.activations import softmax
from keras.callbacks import EarlyStopping, ModelCheckpoint  
from keras.initializers import random_normal, Ones 
from keras.layers import LSTM, Dropout, Input
from keras.models import Model
import keras.backend as K
# SKLearn
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# endregion

Using TensorFlow backend.


In [2]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

current_timestamp = datetime.now().strftime('%d%m%Y_%H%M%S')
if not IN_COLAB:
    
    # region File mount and config
    # drive.mount('/content/gdrive', force_remount=True)
    root_dir = ""
    
    time_dir = os.path.join(root_dir, "result")
    time_dir = os.path.join(time_dir, current_timestamp)
    
    data_dir = root_dir + 'data'
    model_dir = os.path.join(time_dir, 'model')
    plot_dir = os.path.join(time_dir, 'plot')
    result_dir = os.path.join(time_dir, 'result')
    # Create folder if not exists
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
else:
    # region File mount and config
    drive.mount('/content/gdrive', force_remount=True)
    root_dir = "/content/gdrive/My Drive/stock"
    
    time_dir = os.path.join(root_dir, "result")
    
    data_dir = os.path.join(root_dir, "data")
    model_dir = os.path.join(time_dir, 'model')
    plot_dir = os.path.join(time_dir, 'plot')
    result_dir = os.path.join(time_dir, 'result')
    
pd.options.display.max_columns = 12
pd.options.display.max_rows = 24

# disable warnings in Anaconda
warnings.filterwarnings('ignore')

# endregion

Mounted at /content/gdrive


In [0]:
# region Data Loading
stock_name = '000001.SS'  # SSE Composite Index
# df_org = yf.download(stock_name, start="1991-01-01", end="2016-12-31", interval="1wk")
df_org = pd.read_csv(f'{data_dir}/{stock_name}.csv', parse_dates=['Date'])
df_org = df_org.sort_values('Date')
# df_org.to_csv(f'{base_dir}/{stock_name}.csv')
df_org.reset_index(inplace=True)
df_org = df_org[['Date', 'Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']]

# endregion

In [0]:
# region Data ploting
def plot_ohlc(df):
    trace = go.Ohlc(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    increasing=dict(line=dict(color='#58FA58')),
                    decreasing=dict(line=dict(color='#FA5858')))

    layout = {
        'title': f'{stock_name} Historical Price',
        'xaxis': {'title': 'Date',
                  'rangeslider': {'visible': False}},
        'yaxis': {'title': f'Price'}
    }

    data = [trace]

    fig = go.Figure(data=data, layout=layout)
    fig.write_html(os.path.join(plot_dir, '%s_ohlc.html' % (stock_name)), auto_open=False)


plot_ohlc(df_org)
# endregion

In [0]:
# region Create csv result file
# File to save first results
result_save_fname = os.path.join(result_dir, 'result_%s-%s.csv' % (stock_name, current_timestamp))
of_connection = open(result_save_fname, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['stock_name', 'year', 'loss', 'params', 'iteration', 'windows_size', 'train_time'])
of_connection.close()

# Create file to save bayer best
bayer_save_fname = os.path.join(result_dir, 'bayer_best_%s-%s.csv' % (stock_name, current_timestamp))
of_connection = open(bayer_save_fname, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['stock_name', 'year', 'params', 'model_save_location'])
of_connection.close()
# endregion

In [6]:
# region Sample data

df_org.sample(10)

# endregion

,Date,Close,Open,High,Low,Adj Close,Volume
768,2005-09-20,1155.041992,1222.255005,1223.564941,1147.193970,1155.041992,126200
791,2006-02-28,1288.954956,1294.807007,1308.196045,1275.624023,1288.954956,121000
271,1996-03-12,567.869995,563.880005,567.869995,562.299988,567.869995,0
13,1991-04-02,121.089996,121.209999,121.720001,121.089996,121.089996,0
396,1998-08-04,1234.581055,1299.729004,1312.604004,1227.473022,1234.581055,0
198,1994-10-18,785.849976,624.520020,785.849976,624.520020,785.849976,0
923,2008-09-16,2236.409912,2049.812988,2269.732910,1802.331055,2236.409912,356800
1050,2011-02-22,2905.052979,2941.318115,2944.413086,2839.195068,2905.052979,592600
670,2003-11-04,1340.097046,1363.957031,1412.910034,1322.047974,1340.097046,77800
1033,2010-10-26,3054.020996,3065.843994,3073.384033,2955.479980,3054.020996,852000


In [0]:
# region Const
# Declare const
input_col = ['Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']
output_col = ['Close']
time_col = ['Date']

# Input dimension
input_dim = len(input_col)
# Output dimension
output_dim = len(output_col)

# Number of session to prediction as one time
prediction_size = 1
# For each time model is train, the first is display
sample_display_test_size = 5
# Max bayer iteration
bayer_max_evals = 100


# endregion

In [0]:
# region Declare model
# declare model
def softMaxAxis1(x):
    return softmax(x, axis=1)


def get_model(input_dim, window_size, output_dim, lstm_layer_count=5, drop_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(window_size, input_dim), return_sequences=True))
    model.add(Dropout(rate=drop_rate))

    for i in range(lstm_layer_count - 2):
        model.add(LSTM(units=100, return_sequences=True))
        model.add(Dropout(rate=drop_rate))
    
    model.add(LSTM(output_dim, activation=softMaxAxis1))
    opt = optimizers.Adam(lr=0.05, beta_1=0.99, beta_2=0.999)
    model.compile(loss='MAE', optimizer='adam')
    
    return model


# endregion

In [0]:
# region Error metric
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_square_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean((y_true - y_pred) / y_true)


def relative_root_mean_square_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    res = (y_true - y_pred) / y_true
    res = np.power(res, 2)
    res = np.mean(res)
    res = math.sqrt(res)

    return res


# endregion

In [0]:
# region Data preprocessing
# reprocessing data
def next_window(df, i, windows_size, prediction_size, input_col, output_col, time_col):
    '''Generates the next data window from the given index location i'''
    window = df[i: i + windows_size + prediction_size]
    x = window[input_col][:-prediction_size]
    y = window[output_col][-prediction_size:]
    y_time = window[time_col][-prediction_size:]
    return x, y, y_time

def smooting_data(df, window_size):
    return df.ewm(span=window_size).mean()

def preprocessing_data(df, windows_size, prediction_size, input_col, output_col, time_col):
    '''
    Create x, y train data windows
    Warning: batch method, not generative, make sure you have enough memory to
    load data, otherwise use generate_training_window() method.
    '''


    data_x = []
    data_y = []
    data_y_time = []
    for i in range(len(df) - windows_size - prediction_size):
        x, y, y_time = next_window(df, i, windows_size, prediction_size, input_col, output_col, time_col)
        data_x.append(x.values)
        data_y.append(y.values)
        data_y_time.append(y_time)

    time = pd.concat(data_y_time)

    return np.array(data_x), np.array(data_y), time.values


def split_train_test_data(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=False)

    return X_train, y_train, X_valid, y_valid


# endregion

In [0]:
# region Model train
# Trainning model
def train_model(model, X_train, y_train, X_valid, y_valid, stock_name, year, window_size):
    if not IN_COLAB:
        model_save_fname = os.path.join(model_dir, '%s-%s-w%d.h5' % (stock_name, year, window_size))
    else:
        model_save_fname = os.path.join(model_dir, '%s-%s-w%d-%s.h5' % (stock_name, year, window_size, datetime.now().strftime('%d%m%Y_%H%M%S')))
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=100),
        ModelCheckpoint(filepath=model_save_fname, monitor='val_loss', save_best_only=True)
    ]
    history = model.fit(
        X_train,
        y_train,
        epochs=1000,
        batch_size=10000,
        validation_data=(X_valid, y_valid),
        verbose=1,
        callbacks=callbacks,
        shuffle=False)
    model.save(model_save_fname)
    
    return history


# endregion

In [0]:
# region Test model
def test_model(model, test_data, window_size, prediction_size, input_col, output_col, time_col):
    X, y, time = preprocessing_data(test_data, window_size, prediction_size, input_col, output_col, time_col)
    
    y_pred = model.predict(X)
    
    y_pred = np.repeat(y_pred, input_dim, axis=1)
    y_pred = scaler.inverse_transform(y_pred)[:, [0]]
    y_pred = pd.Series(y_pred.flatten())

    df_test_result = pd.DataFrame(time, columns=['Date'])
    df_test_result['Prediction'] = y_pred
    df_test_result.set_index('Date', inplace=True)

    return df_test_result


def plot_test_result(test_result, stock_name, year, window_size):
    # Plotly
    trace0 = go.Scatter(
        x=test_result.index,
        y=test_result['Close'],
        name='Thực tế',
        line=dict(
            color=('#5042f4'),
            width=2)
    )

    trace1 = go.Scatter(
        x=test_result.index,
        y=test_result['Prediction'],
        name='Dự đoán',
        line=dict(
            color=('#005b4e'),
            width=2,
            dash='dot'
        )  # dash options include 'dash', 'dot', and 'dashdot'
    )

    data = [trace0, trace1]

    # Edit the layout
    layout = dict(title='Biểu đồ dự đoán',
                  xaxis=dict(title='Date'),
                  yaxis=dict(title='Price'),
                  paper_bgcolor='#FFF9F5',
                  plot_bgcolor='#FFF9F5'
                  )

    fig = go.Figure(data=data, layout=layout)
    if not IN_COLAB:
        fig.write_html(os.path.join(plot_dir, '%s_%s_w%d.html' % (stock_name, year, window_size)), auto_open=False)
    else:
        fig.write_html(os.path.join(plot_dir, '%s_%s_w%d_%s.html' % (stock_name, year, window_size, datetime.now().strftime('%d%m%Y_%H%M%S'))), auto_open=False)
        
# endregion

In [13]:
# region Bayers
def objective(params, df):
    # Keep track of evals
    global ITERATION

    ITERATION += 1

    # Make sure windows_size is int
    windows_size = int(params['windows_size'])
    print(f'Window size is {windows_size}')

    model = get_model(input_dim, windows_size, output_dim)

    start = timer()

    # Handle data
    # TODO: smoothing ddata
    # df[input_col] = smooting_data(df[input_col], windows_size)

    X, y, time = preprocessing_data(df, windows_size, prediction_size, input_col, output_col, time_col)

    # Reshape data
    y = y.reshape((y.shape[0], y.shape[1]))

    X_train, y_train, X_valid, y_valid = split_train_test_data(X, y)

    # Perform n_train
    history = train_model(model, X_train, y_train, X_valid, y_valid, stock_name, year, windows_size)

    run_time = timer() - start

    # Test generated loss
    test_result = test_model(model, df, windows_size, prediction_size, input_col, output_col, time_col)
    test_result = test_result.join(df_org.set_index('Date'))
    plot_test_result(test_result, stock_name, year, windows_size)

    score = model.evaluate(X, y, 10000, 1)
    print(f'Window size {windows_size} score = {score}')
    #mae = mean_absolute_error(test_result['Close'], test_result['Prediction'])
    #mse = mean_squared_error(test_result['Close'], test_result['Prediction'])
    #mape = mean_absolute_percentage_error(test_result['Close'], test_result['Prediction'])
    #rrmse = relative_root_mean_square_error(test_result['Close'], test_result['Prediction'])

    #print(f'{stock_name} prediction for {prediction_size} day ahead')
    #print(f'MAE = {mae}')
    #print(f'MSE = {mse}')
    #print(f'MAPE = {mape}')
    #print(f'RRMSE = {rrmse}')
    #loss = mape
    loss = score
    # write row
    of_connection = open(result_save_fname, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([stock_name, year, loss, params, ITERATION, windows_size, run_time])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION, 'test_result': test_result,
            'train_time': run_time, 'status': STATUS_OK}

start_year = df_org['Date'].values[:1][0]
start_year = pd.to_datetime(start_year).year

end_year = df_org['Date'].values[-1:][0]
end_year = pd.to_datetime(end_year).year

windows_size_best = []
# Global variable
global ITERATION

for year in range(start_year, end_year + 1):
    df = df_org[df_org['Date'].dt.year == year].copy()

    # Data too small, skip
    if df.shape[0] < 10:
        continue

    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_cols = scaler.fit_transform(df[input_col])
    df[input_col] = scaled_cols

    # Hyperparameter grid
    param_grid = {
        'windows_size': hp.choice('windows_size', np.arange(1, 8, dtype=int))
    }

    bayes_trials = Trials()

    # Create the algorithm
    bayes_algo = tpe.suggest

    ITERATION = 0

    fmin_objective = partial(objective, df=df)
    bayes_best = fmin(fn=fmin_objective, space=param_grid,
                      algo=bayes_algo, trials=bayes_trials,
                      max_evals=bayer_max_evals)

    of_connection = open(bayer_save_fname, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([stock_name, year, bayes_best, '123'])
    of_connection.close()

    windows_size_best.append([year, bayes_best])
# endregion


Window size is 5
  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



,Date,Close,Open,High,Low,Adj Close,Volume
4,1991-01-29,0.223639,0.256090,0.243653,0.249011,0.223639,0.0
45,1991-11-12,1.398926,1.386182,1.398926,1.390844,1.398926,0.0
8,1991-02-26,0.254609,0.288209,0.275045,0.280458,0.254609,0.0
21,1991-05-28,0.085853,0.048394,0.085853,0.063215,0.085853,0.0
50,1991-12-17,1.873275,1.848351,1.873275,1.849503,1.873275,0.0
34,1991-08-27,0.762667,0.711360,0.762667,0.721147,0.762667,0.0
12,1991-03-26,0.135995,0.155529,0.145370,0.154241,0.135995,0.0
10,1991-03-12,0.177921,0.232593,0.220689,0.202589,0.177921,0.0
24,1991-06-18,0.256505,0.213839,0.256505,0.227404,0.256505,0.0
6,1991-02-12,0.260297,0.260401,0.260297,0.273612,0.260297,0.0


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 37 samples, validate on 10 samples
Epoch 1/1000
  0%|          | 0/100 [00:04<?, ?it/s, best loss: ?]WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.





37/37 [==============================]
 - 4s 102ms/step - loss: 0.2942 - val_loss: 1.0737

Epoch 2/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2931 - val_loss: 1.0739

Epoch 3/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2917 - val_loss: 1.0742

Epoch 4/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2902 - val_loss: 1.0743

Epoch 5/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2884 - val_loss: 1.0739

Epoch 6/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2867 - v

,Date,Close,Open,High,Low,Adj Close,Volume
2,1991-01-15,0.278310,0.296939,0.283577,0.304525,0.278310,0.0
4,1991-01-29,0.223639,0.256090,0.243653,0.249011,0.223639,0.0
1,1991-01-08,0.282840,0.274736,0.282840,0.287838,0.282840,0.0
26,1991-07-02,0.289266,0.310088,0.303487,0.315542,0.289266,0.0
40,1991-10-08,0.914885,0.865488,0.914885,0.874104,0.914885,0.0
20,1991-05-21,0.029917,0.017461,0.029917,0.026527,0.029917,0.0
14,1991-04-09,0.115348,0.144751,0.134836,0.139052,0.115348,0.0
12,1991-03-26,0.135995,0.155529,0.145370,0.154241,0.135995,0.0
52,1991-12-31,2.000000,2.000000,2.000000,2.000000,2.000000,0.0
45,1991-11-12,1.398926,1.386182,1.398926,1.390844,1.398926,0.0


Train on 36 samples, validate on 10 samples
Epoch 1/1000
36/36 [==============================]
 - 5s 139ms/step - loss: 0.2807 - val_loss: 1.1119

Epoch 2/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2796 - val_loss: 1.1111

Epoch 3/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2783 - val_loss: 1.1102

Epoch 4/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2769 - val_loss: 1.1088

Epoch 5/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2754 - val_loss: 1.1065

Epoch 6/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2732 - val_loss: 1.1027

Epoch 7/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2709 - val_loss: 1.0966

Epoch 8/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2675 - val_loss: 1.0873

Epoch 9/1000
36/36 [==============================]
 - 0s 2ms/step - loss: 0.2636 - val_loss: 1.0734

Epoch 10/1000
36/36 [===============

...

,Date,Close,Open,High,Low,Adj Close,Volume
25,1991-06-25,0.305804,0.283466,0.313283,0.296502,0.305804,0.0
40,1991-10-08,0.914885,0.865488,0.914885,0.874104,0.914885,0.0
23,1991-06-11,0.189087,0.169217,0.189087,0.183121,0.189087,0.0
20,1991-05-21,0.029917,0.017461,0.029917,0.026527,0.029917,0.0
15,1991-04-16,0.092805,0.120177,0.110818,0.116162,0.092805,0.0
16,1991-04-23,0.073001,0.097004,0.088170,0.096053,0.073001,0.0
31,1991-08-06,0.502686,0.457318,0.502686,0.469034,0.502686,0.0
32,1991-08-13,0.583483,0.537616,0.583483,0.548722,0.583483,0.0
8,1991-02-26,0.254609,0.288209,0.275045,0.280458,0.254609,0.0
0,1991-01-01,0.255346,0.233348,0.255346,0.246764,0.255346,0.0


Train on 37 samples, validate on 10 samples
Epoch 1/1000
37/37 [==============================]
 - 13s 362ms/step - loss: 0.3202 - val_loss: 1.0141

Epoch 2/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3187 - val_loss: 1.0156

Epoch 3/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3171 - val_loss: 1.0173

Epoch 4/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3155 - val_loss: 1.0189

Epoch 5/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3130 - val_loss: 1.0206

Epoch 6/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3104 - val_loss: 1.0222

Epoch 7/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3075 - val_loss: 1.0233

Epoch 8/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.3039 - val_loss: 1.0235

Epoch 9/1000
37/37 [==============================]
 - 0s 2ms/step - loss: 0.2986 - val_loss: 1.0222

Epoch 10/1000
37/37 [==============

,Date,Close,Open,High,Low,Adj Close,Volume
0,1991-01-01,0.255346,0.233348,0.255346,0.246764,0.255346,0.0
29,1991-07-23,0.354998,0.347058,0.354998,0.359611,0.354998,0.0
52,1991-12-31,2.000000,2.000000,2.000000,2.000000,2.000000,0.0
28,1991-07-16,0.318656,0.294029,0.318656,0.306985,0.318656,0.0
19,1991-05-14,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,1991-02-12,0.260297,0.260401,0.260297,0.273612,0.260297,0.0
18,1991-05-07,0.000211,0.039125,0.031602,0.022141,0.000211,0.0
10,1991-03-12,0.177921,0.232593,0.220689,0.202589,0.177921,0.0
2,1991-01-15,0.278310,0.296939,0.283577,0.304525,0.278310,0.0
24,1991-06-18,0.256505,0.213839,0.256505,0.227404,0.256505,0.0


Train on 39 samples, validate on 10 samples
Epoch 1/1000
39/39 [==============================]
 - 16s 399ms/step - loss: 0.3511 - val_loss: 0.9504

 11%|█         | 11/100 [06:51<1:03:48, 43.01s/it, best loss: 0.1557055115699768]


KeyboardInterrupt: ignored